<a href="https://colab.research.google.com/github/agemagician/CodeTrans/blob/main/prediction/single%20task/source%20code%20summarization/csharp/small_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**<h3>Summarize the csharp source code using codeTrans single task training model</h3>**
<h4>You can make free prediction online through this 
<a href="https://huggingface.co/SEBIS/code_trans_t5_small_source_code_summarization_csharp">Link</a></h4> (When using the prediction online, you need to parse and tokenize the code first.)

**1. Load necessry libraries including huggingface transformers**

In [ ]:
!pip install -q transformers sentencepiece

     |████████████████████████████████| 1.4MB 6.0MB/s 
     |████████████████████████████████| 1.1MB 51.3MB/s 
     |████████████████████████████████| 2.9MB 55.3MB/s 
     |████████████████████████████████| 890kB 53.9MB/s 


In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline

**2. Load the token classification pipeline and load it into the GPU if avilabile**

In [ ]:
pipeline = SummarizationPipeline(
    model=AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_small_source_code_summarization_csharp"),
    tokenizer=AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_small_source_code_summarization_csharp", skip_special_tokens=True),
    device=0
)

/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_auto.py:852: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


**3 Give the code for summarization, parse and tokenize it**

In [ ]:
code = "public static DateTime ParseUnixDateTime(double unixTime)\n    {\n        var dt= new DateTime(1970, 1, 1, 0, 0, 0, 0, System.DateTimeKind.Utc);\n        dt= dt.AddSeconds(unixTimeStamp).ToLocalTime();\n        return dt;\n    }" #@param {type:"raw"}

In [ ]:
!pip install antlr4-python3-runtime==4.5.2
import antlr4

!wget https://www.dropbox.com/s/o87gk1jxf8645eu/CSharp4Lexer.py?dl=1 -O CSharp4Lexer.py
from CSharp4Lexer import CSharp4Lexer

     |████████████████████████████████| 112kB 4.3MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.5.2-cp36-none-any.whl size=165556 sha256=aa0e68724ec56fa830cf741ba3ec991707a9453151f0e6ce1cea7fe1243a4dd0
  Stored in directory: /root/.cache/pip/wheels/e8/8f/2d/020b9595d9afea75db47d851fea1812250bae782bba902c72b
Successfully built antlr4-python3-runtime
--2020-12-07 14:27:17--  https://www.dropbox.com/s/o87gk1jxf8645eu/CSharp4Lexer.py?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.1, 2620:100:601d:1::a27d:501
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/o87gk1jxf8645eu/CSharp4Lexer.py [following]
--2020-12-07 14:27:17--  https://www.dropbox.com/s/dl/o87gk1jxf8645eu/CSharp4Lexer.py
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce85a4a98814f0a6cdf62aae26c.dl

In [ ]:
def csTokenizer(line):
    l = line.replace('\\n', '\n')
    parsedVersion = []
    stream = antlr4.InputStream(l)
    lexer = CSharp4Lexer(stream)
    toks = antlr4.CommonTokenStream(lexer)
    toks.fetch(500)

    identifiers = {}
    identCount = 0
    for token in toks.tokens:
      if token.type == 109:
        parsedVersion.append("CODE_INTEGER")
      elif token.type == 111:
        parsedVersion.append("CODE_REAL")
      elif token.type == 112:
        parsedVersion.append("CODE_CHAR")
      elif token.type == 113:
        parsedVersion.append("CODE_STRING")
      elif token.type == 9 or token.type == 7 or token.type == 6 or token.type == 4 or token.type == 8 or token.type == 5: # whitespace and comments and newline
        pass
      else:
        parsedVersion.append(str(token.text))

    parsedVersion.remove('<EOF>')
    return ' '.join(parsedVersion)

tokenized_code = csTokenizer(code)
print("code after tokenization: " + tokenized_code)

code after tokenization: public static DateTime ParseUnixDateTime ( double unixTime ) { var dt = new DateTime ( CODE_INTEGER , CODE_INTEGER , CODE_INTEGER , CODE_INTEGER , CODE_INTEGER , CODE_INTEGER , CODE_INTEGER , System . DateTimeKind . Utc ) ; dt = dt . AddSeconds ( unixTimeStamp ) . ToLocalTime ( ) ; return dt ; }


**4. Make Prediction**

In [ ]:
pipeline([tokenized_code])

Your max_length is set to 512, but you input_length is only 85. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


[{'summary_text': 'How to convert C # DateTime to C #'}]